# Importation des bibliotheques

In [15]:
import os
import re
from collections import defaultdict
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import torch
import torch.nn as nn
import torchvision.transforms as transforms
import torchvision.datasets as datasets
from torch.utils.data import DataLoader
from timm import create_model
import time 
import json
import splitfolders
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix, classification_report
import numpy as np

### Ici on va entrainer notre modele sur notre dataset d'entrainement complet

In [2]:
# Chemin du dossier train_data
dossier_train_data = r"D:\african_plums_dataset\african_plums_Splits\train"

# Organiser les images en sous-dossiers
for file_name in os.listdir(dossier_train_data):
    # Utiliser une expression régulière pour extraire la classe
    match = re.match(r"^(.*?)(\d+)\.jpeg$", file_name)
    if match:
        class_name = match.group(1)
        class_dir = os.path.join(dossier_train_data, class_name)
        
        # Créer le sous-dossier si nécessaire
        if not os.path.exists(class_dir):
            os.makedirs(class_dir)
        
        # Déplacer l'image dans le sous-dossier
        src_path = os.path.join(dossier_train_data, file_name)
        dst_path = os.path.join(class_dir, file_name)
        shutil.move(src_path, dst_path)

print("Images réorganisées en sous-dossiers par classe.")


Images réorganisées en sous-dossiers par classe.


In [3]:
# Les données d'entrainement et de test
dossier_train = r"D:\african_plums_dataset\african_plums_Splits\train"

# Définitions des paramètres pour la taille des images et le nombres d'images traités par lot
IMG_SIZE = (128, 128)
BATCH_SIZE = 32

# Créer un dataset à partir des fichiers d'entraînement
train_dataset = tf.keras.preprocessing.image_dataset_from_directory(
    directory=dossier_train,
    labels='inferred', # Pour specifier que chaque images de chaque sous repertoires appartient uniquement a cette classe
    label_mode='int', # Encodage en entier pour convertir les labels(classes/sous repertoire) en entier afin de faciliter l'entrainement
    batch_size=BATCH_SIZE,
    image_size=IMG_SIZE,
    shuffle=True
)

# Normaliser les images
normalization_layer = tf.keras.layers.Rescaling(1./255)

# Appliquer la normalisation aux datasets
train_dataset = train_dataset.map(lambda x, y: (normalization_layer(x), y))

# Afficher la structure des datasets
for images, labels in train_dataset.take(1):
    print(images.shape)  # Afficher la forme des tenseurs d'images
    print(labels.shape)  # Afficher la forme des tenseurs de labels


Found 3603 files belonging to 6 classes.
(32, 128, 128, 3)
(32,)


### Application de la data augmentation

Grace a cette technique, lors de l'entrainement du modele sur 10 epoques, le modele aura un equivalent d'environ 30000 images dans sa base d'entrainement. Le principe est le suivant: Lors de l'entrainement du modele sur chaque époques, chaque image recevra les transformations suivante avant d'etre reconnu par le modele comme element d'une classe.

In [4]:
# Chemin des dossiers train et test
dossier_train = r"D:\african_plums_dataset\african_plums_Splits\train"

# Créer un générateur d'images avec augmentation pour l'entraînement
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

# Appliquer le générateur aux données d'entraînement
train_generator = train_datagen.flow_from_directory(
    directory=dossier_train,
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='sparse'  # Utiliser 'sparse' pour les étiquettes entières
)

# Afficher la structure des datasets
for images, labels in train_generator:
    print(images.shape)  # Afficher la forme des tenseurs d'images
    print(labels.shape)  # Afficher la forme des tenseurs de labels
    break  # Pour éviter d'afficher toutes les images


Found 3603 images belonging to 6 classes.
(32, 128, 128, 3)
(32,)


### Vérifieons le contenu du répertoire train avant son passage dans le modele et voyons a quoi correspond chaque classe de l'ensemble d'entrainement par rapport aux classes de depart. On va verifier cela avec les indices car plus haut nous avons utiliser un encodage pour encoder nos differentes classes en valeurs entieres (le target encoding) pour faciliter le passage dans le modele


In [5]:
# Chemin du dossier d'entrainement
dossier_train = r"D:\african_plums_dataset\african_plums_Splits\train"
# Affichage des classes
print("Contenu du répertoire 'train_data' :")
for file_name in os.listdir(dossier_train):
    print(file_name)


Contenu du répertoire 'train_data' :
bruised
cracked
rotten
spotted
unaffected
unripe


In [6]:
# Récupérer les noms des classes
class_names = train_generator.class_indices
class_names = {v: k for k, v in class_names.items()}  # Inverser le dictionnaire pour obtenir les noms des classes

# Affichage des noms des classes 
print("Classes et indices correspondants :")
for idx, class_name in class_names.items():
    print(f"Indice {idx} : {class_name}")


Classes et indices correspondants :
Indice 0 : bruised
Indice 1 : cracked
Indice 2 : rotten
Indice 3 : spotted
Indice 4 : unaffected
Indice 5 : unripe


In [7]:
# Chemin du dataset
data_dir = r'D:\african_plums_dataset\african_plums_Splits\train'

# Préparer les transformations et le DataLoader
transform = transforms.Compose([
    transforms.Resize((224, 224)), 
    transforms.ToTensor(), # Ici on normalise les images en les fesant passer de la plage [0,255] en la plage [0,1] car initialement les images sont des tenseurs numpy
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]), # Normalisation appliqué au format des couleurs RGB pour une moyenne de mean et un ecart type de std
]) # On aurait pu ajouter d'autre transformation mais ca serait une redondance car cela a deja été fait dans notre section data augmentation.

# Application de ce second prétraitement aux données de la base train_data
train_dataset = datasets.ImageFolder(root=data_dir, transform=transform)
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)

# Charger le modèle ViT pré-entraîné depuis timm avec 6 sorties correspondant au 6 classes
model = create_model('vit_base_patch16_224', pretrained=True)
model.head = nn.Linear(model.head.in_features, 6)

# Définir l'optimiseur et la fonction de perte
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-4)


In [12]:
# Entrainement du model avec affichage de la précision et de la perte
for epoch in range(10):  
    start_time = time.time() # Début de l'époque
    correct_train = 0 # Nombre total d'image correctement predit dans cette epoque
    total_train = 0 # Nombre total d'image vu dans cette epoque
    for images, labels in train_loader:
        optimizer.zero_grad() # Reinitialisation des gradients
        outputs = model(images) # Predictions
        loss = criterion(outputs, labels) # Loss
        loss.backward() # Retropropagation
        optimizer.step() # Ajustement des parametres
        
        _, predicted = torch.max(outputs.data, 1) # On reccupere l'indice de la classe predite avec la plus grande probabilité
        total_train += labels.size(0) # Mise a jour du nombre d'image vu dans cette epoque jusqu'a cette etapes
        correct_train += (predicted == labels).sum().item() # Mise a jour du Nombre total d'image correctement predit dans cette epoque
    
    train_accuracy = 100 * correct_train / total_train # Calcul de la precision apres chaque etapes
    end_time = time.time() # Fin de l'époque 
    epoch_time = end_time - start_time # Calcul du temps d'exécution 
    print(f"Epoch [{epoch+1}/10], Loss: {loss.item():.4f}, Train Accuracy: {train_accuracy:.2f}%, Time: {epoch_time:.2f} seconds")
    #print(f"Epoch [{epoch+1}/10], Loss: {loss.item():.4f}, Train Accuracy: {train_accuracy:.2f}%") # affichage de l'epoque, de la perte et de la precision


Epoch [1/10], Loss: 0.3671, Train Accuracy: 87.79%, Time: 4266.12 seconds
Epoch [2/10], Loss: 0.1547, Train Accuracy: 88.79%, Time: 4258.68 seconds
Epoch [3/10], Loss: 0.6418, Train Accuracy: 89.20%, Time: 4382.40 seconds
Epoch [4/10], Loss: 0.1674, Train Accuracy: 89.65%, Time: 4507.09 seconds
Epoch [5/10], Loss: 0.0738, Train Accuracy: 89.73%, Time: 4588.03 seconds
Epoch [6/10], Loss: 0.2275, Train Accuracy: 88.98%, Time: 4552.62 seconds
Epoch [7/10], Loss: 0.1013, Train Accuracy: 89.56%, Time: 4434.91 seconds
Epoch [8/10], Loss: 0.0967, Train Accuracy: 90.23%, Time: 4288.01 seconds
Epoch [9/10], Loss: 0.0168, Train Accuracy: 89.18%, Time: 10444.49 seconds
Epoch [10/10], Loss: 0.2285, Train Accuracy: 89.79%, Time: 4360.15 seconds


## Evaluation du  model

In [13]:
# Chemin du dataset de test
test_dir = r'D:\african_plums_dataset\african_plums_Splits\test'

# Utiliser les mêmes transformations que pour l'entraînement
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

# Charger le dataset de test
test_dataset = datasets.ImageFolder(root=test_dir, transform=transform)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)  # shuffle=False pour l'évaluation

In [14]:
model.eval()  # Mode évaluation
correct_test = 0
total_test = 0
test_loss = 0.0

with torch.no_grad():  # Désactive le calcul des gradients
    for images, labels in test_loader:
        outputs = model(images)
        loss = criterion(outputs, labels)
        test_loss += loss.item()
        
        _, predicted = torch.max(outputs.data, 1)
        total_test += labels.size(0)
        correct_test += (predicted == labels).sum().item()

test_accuracy = 100 * correct_test / total_test
test_loss = test_loss / len(test_loader)  # Perte moyenne par batch

print(f"Test Loss: {test_loss:.4f}, Test Accuracy: {test_accuracy:.2f}%")

Test Loss: 1.6705, Test Accuracy: 69.91%


In [16]:
all_preds = []
all_labels = []

with torch.no_grad():
    for images, labels in test_loader:
        outputs = model(images)
        _, preds = torch.max(outputs, 1)
        all_preds.extend(preds.cpu().numpy())
        all_labels.extend(labels.cpu().numpy())

# Matrice de confusion
print("Matrice de confusion:")
print(confusion_matrix(all_labels, all_preds))

# Rapport de classification (précision, rappel, F1-score)
print("\nRapport de classification:")
print(classification_report(all_labels, all_preds, target_names=test_dataset.classes))

Matrice de confusion:
[[ 13   5  16  10  12   8]
 [  3  16   5   4   3   2]
 [ 17   4 108   5   1   9]
 [ 11   0   9  75  39  18]
 [  2   1   3  29 296  14]
 [  2   0  11  17  12 124]]

Rapport de classification:
              precision    recall  f1-score   support

     bruised       0.27      0.20      0.23        64
     cracked       0.62      0.48      0.54        33
      rotten       0.71      0.75      0.73       144
     spotted       0.54      0.49      0.51       152
  unaffected       0.82      0.86      0.84       345
      unripe       0.71      0.75      0.73       166

    accuracy                           0.70       904
   macro avg       0.61      0.59      0.60       904
weighted avg       0.69      0.70      0.69       904



In [18]:
torch.save(model, "model_full2.pth")

In [17]:
torch.save(model.state_dict(), "model2.pth")

In [10]:
#class_indices = train_generator.class_indices
#with open("class_indices.json", "w") as f:
 #   json.dump(class_indices, f)

## Evaluation du premier model

In [20]:
# --- Configuration ---
TEST_DIR = r"D:\african_plums_dataset\african_plums_Splits\test"  # Chemin vers vos données de test
MODEL_PATH = r"C:\Users\Christian\Desktop\PrunesCheck\model1.pth"  # Chemin vers votre modèle sauvegardé
IMG_SIZE = 224
BATCH_SIZE = 32
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")


In [21]:
# --- 1. Préparation des données de test ---
transform = transforms.Compose([
    transforms.Resize((IMG_SIZE, IMG_SIZE)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])


In [22]:
test_dataset = datasets.ImageFolder(TEST_DIR, transform=transform)
test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False)

# --- 2. Rechargement du modèle ---
model = create_model('vit_base_patch16_224', pretrained=False)  # Même architecture qu'à l'entraînement
model.head = nn.Linear(model.head.in_features, 6)  # Adapté à vos 6 classes
model.load_state_dict(torch.load(MODEL_PATH))  # Chargement des poids
model = model.to(DEVICE)
model.eval()  # Mode évaluation


C:\Users\Christian\AppData\Local\Temp\ipykernel_7180\2462356336.py:7: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load(MODEL_PATH))  # Chargeme

VisionTransformer(
  (patch_embed): PatchEmbed(
    (proj): Conv2d(3, 768, kernel_size=(16, 16), stride=(16, 16))
    (norm): Identity()
  )
  (pos_drop): Dropout(p=0.0, inplace=False)
  (patch_drop): Identity()
  (norm_pre): Identity()
  (blocks): Sequential(
    (0): Block(
      (norm1): LayerNorm((768,), eps=1e-06, elementwise_affine=True)
      (attn): Attention(
        (qkv): Linear(in_features=768, out_features=2304, bias=True)
        (q_norm): Identity()
        (k_norm): Identity()
        (attn_drop): Dropout(p=0.0, inplace=False)
        (proj): Linear(in_features=768, out_features=768, bias=True)
        (proj_drop): Dropout(p=0.0, inplace=False)
      )
      (ls1): Identity()
      (drop_path1): Identity()
      (norm2): LayerNorm((768,), eps=1e-06, elementwise_affine=True)
      (mlp): Mlp(
        (fc1): Linear(in_features=768, out_features=3072, bias=True)
        (act): GELU(approximate='none')
        (drop1): Dropout(p=0.0, inplace=False)
        (norm): Identity(

In [23]:
# --- 3. Évaluation ---
all_preds = []
all_labels = []
all_probs = []

with torch.no_grad():
    for images, labels in test_loader:
        images, labels = images.to(DEVICE), labels.to(DEVICE)
        outputs = model(images)
        probs = torch.softmax(outputs, dim=1)
        _, preds = torch.max(outputs, 1)
        
        all_preds.extend(preds.cpu().numpy())
        all_labels.extend(labels.cpu().numpy())
        all_probs.extend(probs.cpu().numpy())

In [24]:
# --- 4. Métriques de performance ---
print("Rapport de classification :")
print(classification_report(all_labels, all_preds, target_names=test_dataset.classes))

print("\nMatrice de confusion :")
print(confusion_matrix(all_labels, all_preds))

# --- 5. (Optionnel) Analyse par classe ---
for i, class_name in enumerate(test_dataset.classes):
    class_idx = np.where(np.array(all_labels) == i)[0]
    class_acc = np.mean(np.array(all_preds)[class_idx] == i)
    print(f"Précision pour {class_name}: {class_acc:.2%}")o

Rapport de classification :
              precision    recall  f1-score   support

     bruised       0.23      0.27      0.25        64
     cracked       0.54      0.45      0.49        33
      rotten       0.72      0.76      0.74       144
     spotted       0.51      0.32      0.39       152
  unaffected       0.76      0.89      0.82       345
      unripe       0.77      0.72      0.74       166

    accuracy                           0.68       904
   macro avg       0.59      0.57      0.57       904
weighted avg       0.67      0.68      0.67       904


Matrice de confusion :
[[ 17   6  17   4  12   8]
 [  3  15   4   3   6   2]
 [ 20   5 109   3   2   5]
 [ 20   0  13  48  58  13]
 [  8   1   1  20 308   7]
 [  6   1   7  16  17 119]]
Précision pour bruised: 26.56%
Précision pour cracked: 45.45%
Précision pour rotten: 75.69%
Précision pour spotted: 31.58%
Précision pour unaffected: 89.28%
Précision pour unripe: 71.69%
